In [32]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle 
import pandas as pd
import numpy as np

In [33]:
### Load all models - trained model, scaler, onehot encoder
model=load_model('model.h5')

# Load encoder and scaler
with open('onehot_encode_geo.pkl', 'rb') as file:
    lable_encoder_geo=pickle.load(file)
    
with open('label_encoder_gender.pkl', 'rb') as file:
    lable_encoder_gender=pickle.load(file)
    
with open('scaler.pkl', 'rb') as file:
    scaler=pickle.load(file)

In [34]:
# exmple inpt data
input_data={
    'CreditScore': 600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':500000
}

In [35]:
type(lable_encoder_gender)

sklearn.preprocessing._label.LabelEncoder

In [36]:
geo_encoded = lable_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=lable_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

e:\EndToEnd_DL_Project\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [37]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,500000


In [38]:
## encode categorycal variable gender
input_df['Gender']=lable_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,500000


In [39]:
input_df=pd.concat([input_df, geo_encoded_df], axis=1)
input_df.drop('Geography', axis=1, inplace=True)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,500000,1.0,0.0,0.0


In [40]:
## Scaling of input 
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.51732154,  0.91094795,  0.10104752, -0.6945015 , -0.26331625,
         0.79989064,  0.64550866,  0.96704278,  6.95392421,  0.99600797,
        -0.57831252, -0.57330877]])

In [41]:
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 186ms/step


array([[0.19130889]], dtype=float32)

In [42]:
prediction_prob=prediction[0][0]
prediction_prob

0.19130889

In [43]:
if prediction_prob> 0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
